In [ ]:
# Bosques Aleatorios
#### 0.1.1 - 2025 - 01 - 01
#### Dr. MArco Aceves 
#### rev en Jupyter Notebook
#### Código como ejemplo como parte del libro:
#### de 0 a 100 en IA
#### 4_Bosques.ipynb

Como ya hemos visto, los bosques aleatorios son un algoritmo utilizado para problemas de clasificación y regresión. Este algoritmo crea subconjuntos de datos aleatorios a partir del conjunto de datos original y con cada uno de esos subconjuntos crea un árbol cuyos nodos selecciona de manera aleatoria también.

Para ejemplicar este algoritmo, haremos una clasificación de los pasajeros del Titanic, los clasificaremos en supervivientes y no supervivientes.

Para la construcción del algoritmo usaremos la librería **scikit-learn (sklearn)**. Especificamente, la clase **RandomForestClassifier**. Esta clase trabaja únicamente con valores numéricos (*sklearn* también cuenta con una clase para realizar modelos de regresión que se llama *RandomForestRegressor*).

Si no tenemos instalado **scikit-learn** podemos usar el siguiente comando:<br>
pip install -U scikit-learn

Ahora procedemos a realizar el código.

Importamos las librerías.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

Abrimos el archivo que contiene los datos del Titanic.

In [2]:
df= pd.read_csv("titanic.csv")
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Como podemos observar no todas las columnas son relevantes para nuestro propósito, por lo que vamos a seleccionar solo los atributos que nos interesan, los cuales son: *Survived, Pclass, Sex, Age, SibSp, Parch* y *Embarked*.

In [5]:
df = df[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]
df.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,1,1,female,29.0000,0,0,S
1,1,1,male,0.9167,1,2,S
2,0,1,female,2.0000,1,2,S
3,0,1,male,30.0000,1,2,S
4,0,1,female,25.0000,1,2,S


Vemos que tenemos dos atributos no numéricos, por lo que debemos convertirlos para poder utilizar **RandomForestClassifier**. La conversión la podemos hacer con una función de **pandas** que se llama **get_dummies**, **get_dummies** nos devuelve columnas cualitativas convertidas en columnas numéricas con 0 y 1. Esta función recibe como parámetros: el *dataframe*, las columnas sobre las cuales se hará la conversión, a modo de lista, y vamos a mandar verdadero a la variable **drop_first** que es falsa por defecto. Si dejamos **drop_first** como falso tendríamos una columna por cada observación, por ejemplo en el caso de la columna **Sex**, tendríamos una columna llamada *Sex_male* y otra llamada *Sex_female*. En cambio al volver verdadero a **drop_first** vamos a tener una sola columna llamada *Sex_male* donde se usa 1 para hombre y 0 para mujer.

In [6]:
df = pd.get_dummies(df, columns=["sex", "embarked"], drop_first=True)
df.head()

,survived,pclass,age,sibsp,parch,sex_male,embarked_Q,embarked_S
0,1,1,29.0000,0,0,0,0,1
1,1,1,0.9167,1,2,1,0,1
2,0,1,2.0000,1,2,0,0,1
3,0,1,30.0000,1,2,1,0,1
4,0,1,25.0000,1,2,0,0,1


Ya que todos nuestros columnas tienen valores numéricos, procedemos a verificar si existen valores nulos. Lo realizamos con la función **info** de **pandas**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    1309 non-null   int64  
 1   pclass      1309 non-null   int64  
 2   age         1046 non-null   float64
 3   sibsp       1309 non-null   int64  
 4   parch       1309 non-null   int64  
 5   sex_male    1309 non-null   uint8  
 6   embarked_Q  1309 non-null   uint8  
 7   embarked_S  1309 non-null   uint8  
dtypes: float64(1), int64(4), uint8(3)
memory usage: 55.1 KB


En este archivo, tenemos 177 valores nulos, los cuales eliminaremos por practicidad de este ejemplo. Eliminamos los valores nulos con **dropna**.

In [8]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    1046 non-null   int64  
 1   pclass      1046 non-null   int64  
 2   age         1046 non-null   float64
 3   sibsp       1046 non-null   int64  
 4   parch       1046 non-null   int64  
 5   sex_male    1046 non-null   uint8  
 6   embarked_Q  1046 non-null   uint8  
 7   embarked_S  1046 non-null   uint8  
dtypes: float64(1), int64(4), uint8(3)
memory usage: 52.1 KB


Ahora, ya terminamos de preparar nuestros datos y estamos listos para implementar el bosque aleatorio.

Para la construcción de nuestro bosque, vamos a implementar la validación cruzada *k-fold* vista anteriormente, así que importamos nuestras funciones de la validación.

In [9]:
def division(df, k):
    datos = []
    tamano = int(len(df)/k)
    for i in range(k-1):
        division = np.array(range(i*tamano, (i+1)*tamano))
        datos.append(np.array(division))
    datos.append(np.array(range((i+1)*tamano, len(df))))
    return datos


# se filtran los datos para tener entrenamiento y prueba
def conjunto(df, datos, indice):
    prueba = df.iloc[datos[indice]]
    ind = np.array(range(len(df)))
    ind_ent = np.delete(ind, datos[indice])
    entrenamiento = df.iloc[ind_ent]
    return entrenamiento, prueba

Creamos una función en donde ejecutaremos la validación *k-fold* e iremos creando un bosque por cada subconjunto que tengamos, esta función, **crear_bosque**, recibe como parámetros: un *dataframe* **df**, el número de particiones en los que se dividirán los datos **k**, el número de estimadores **estimadores**, el número mínimo de hojas **hojas**, la semilla para el estado aleatorio **semilla** y el nombre de la columna donde se encuentra la salida o variable dependiente **salida**.

Dentro de la función, inicializaremos una variable precisión donde iremos sumando la precisión de cada bosque para poder obtener la media, una vez que termine con los diferentes bosques. 

Posteriormente, llamamos a división para dividir nuestros datos e implementamos un ciclo en el cual, cada iteración corresponderá a un bosque aleatorio. En el ciclo obtenemos los datos de entrenamiento y de prueba, separamos la salida de los demás atributos (requisito para poder usar la clase: RandomForestClassifier), para eliminar una columna de un dataframe usamos la función: drop. 

Esta función nos permite eliminar filas o columnas: axis=0 para indicar que vamos a eliminar filas y axis=1 para indicar que se trata de columnas. También, hay que mandarle el índice de las filas o el nombre de las columnas a eliminar en forma de lista. 

Ya que hemos separado la salida de los demás atributos, instanciamos: RandomForestClassifier mandándole como parámetros, el número de estimadores (número de árboles), la semilla para inicializar el estado aleatorio (solo en la construcción del primer árbol) y el número mínimo de hojas de los árboles. Entrenamos el modelo llamando: fit, mandándole los datos de entrenamiento (los atributos y la salida), y finalmente probamos el bosque con los datos de prueba, esto lo hacemos con la función score, la cual recibe como parámetros los datos de prueba y nos devuelve la precisión del árbol como un número entre 0 y 1. Recordemos que por cada subconjunto creado por la validación cruzada “k-fold”, estamos creando un bosque y vamos a retornar el promedio de la precisión de todos.

In [10]:
def crear_bosque(df, k, estimadores, hojas, semilla, salida):
    # inicializamos la variable de la precisión
    precision = 0.0
    # hacemos las divisiones del conjunto de datos
    datos = division(df, k)
    # por cada subconjunto se crea un bosque y luego promediamos la precisión
    for i in range(k):
        
        # obtenemos los datos de entrenamiento y prueba
        entrenamiento, prueba = conjunto(df, datos, i)
        
        # separamos la salida de los demas atributos
        entrenamiento_atributos = entrenamiento.drop([salida], axis=1)
        entrenamiento_salida = entrenamiento[salida]
        prueba_atributos = prueba.drop([salida], axis=1)
        prueba_salida = prueba[salida]
        
        # damos los hiperparámetros del bosque
        
        # establecemos la semilla en la primera iteración
        if i == 0:
            bosque = RandomForestClassifier(n_estimators=estimadores, random_state=semilla, min_samples_leaf=hojas)
        else:
            bosque = RandomForestClassifier(n_estimators=estimadores, min_samples_leaf=hojas)
        # entrenamos el bosque
        bosque.fit(entrenamiento_atributos, entrenamiento_salida)
        # probamos el bosque y sumamos la precision obtenida
        precision += bosque.score(prueba_atributos, prueba_salida)
    # retornamos el promedio de la precisión obtenida
    return round(precision/k, 3)
        

Ahora, vamos a probar nuestro código y crear bosques con distinto número de árboles, mientras los demás parámetros los mantenemos igual (k, estimadores, semilla, número de hojas, y la salida que se desee).

Un árbol

In [11]:
crear_bosque(df, 5, 1, 8, 1, "survived")

0.76

Cinco árboles

In [12]:
crear_bosque(df, 5, 5, 8, 1, "survived")

0.793

Diez árboles

In [14]:
crear_bosque(df, 5, 10, 8, 1, "survived")

0.798

Como podemos observar, a medida que incrementamos el número de árboles, incrementa la precisión del bosque.